In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1725462091605_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark = SparkSession.builder.appName("RedfinDataAnalysis").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
redfin_data = spark.read.csv("s3://rawdata-redfin/city_market_tracker.tsv000.gz", header=True, inferSchema=True, sep= "\t")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
redfin_data.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------+---------------+-----------+--------------+--------+----------------------+--------------+----------+----------+----------+--------------------+----------------+-----------------+---------------------+---------------------+-----------------+---------------------+---------------------+-----------------+--------------------+--------------------+------------------+--------------------+--------------------+----------+--------------+--------------+-------------+-----------------+-----------------+------------+------------------+------------------+---------+-------------------+-------------------+----------------+--------------------+--------------------+----------+--------------+--------------+------------------+--------------------+--------------------+---------------+-------------------+-------------------+-----------+---------------+--------------------+-----------------------+---------------------------+---------------------------+-------------------+-----------

In [7]:
#Check the schema
redfin_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- period_begin: date (nullable = true)
 |-- period_end: date (nullable = true)
 |-- period_duration: integer (nullable = true)
 |-- region_type: string (nullable = true)
 |-- region_type_id: integer (nullable = true)
 |-- table_id: integer (nullable = true)
 |-- is_seasonally_adjusted: string (nullable = true)
 |-- region: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- property_type: string (nullable = true)
 |-- property_type_id: integer (nullable = true)
 |-- median_sale_price: double (nullable = true)
 |-- median_sale_price_mom: double (nullable = true)
 |-- median_sale_price_yoy: double (nullable = true)
 |-- median_list_price: double (nullable = true)
 |-- median_list_price_mom: double (nullable = true)
 |-- median_list_price_yoy: double (nullable = true)
 |-- median_ppsf: double (nullable = true)
 |-- median_ppsf_mom: double (nullable = true)
 |-- median_ppsf_yoy: double (nul

In [8]:
#print column names
redfin_data.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['period_begin', 'period_end', 'period_duration', 'region_type', 'region_type_id', 'table_id', 'is_seasonally_adjusted', 'region', 'city', 'state', 'state_code', 'property_type', 'property_type_id', 'median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy', 'median_list_price', 'median_list_price_mom', 'median_list_price_yoy', 'median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf', 'median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold', 'homes_sold_mom', 'homes_sold_yoy', 'pending_sales', 'pending_sales_mom', 'pending_sales_yoy', 'new_listings', 'new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom', 'inventory_yoy', 'months_of_supply', 'months_of_supply_mom', 'months_of_supply_yoy', 'median_dom', 'median_dom_mom', 'median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom', 'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom', 'sold_above_list_yoy', 'price_drops', 'price_drops_mom', 'price_drops_yoy', 'off_market_in_two_wee

In [9]:
df_redfin = redfin_data.select(['period_end','period_duration', 'city', 'state', 'property_type',
    'median_sale_price', 'median_ppsf', 'homes_sold', 'inventory', 'months_of_supply', 'median_dom', 'sold_above_list', 'last_updated'])
df_redfin.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+----------+----------+--------------------+-----------------+-----------------+----------+---------+----------------+----------+---------------+-------------------+
|period_end|period_duration|      city|     state|       property_type|median_sale_price|      median_ppsf|homes_sold|inventory|months_of_supply|median_dom|sold_above_list|       last_updated|
+----------+---------------+----------+----------+--------------------+-----------------+-----------------+----------+---------+----------------+----------+---------------+-------------------+
|2019-07-31|             30|  Elbridge|  New York|Single Family Res...|         152000.0|96.08091024020227|         1|        4|             4.0|        10|            1.0|2024-08-14 14:22:06|
|2012-01-31|             30|    Maumee|      Ohio|Single Family Res...|          75605.0|55.33251770348885|         6|       89|            14.8|        80|            0.0|2024-08-14 14:22:06|
|2013-12-31|             30|Clear L

In [10]:
#check total number of rows
print(f"Total number of rows: {df_redfin.count()}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total number of rows: 5522511

In [11]:
from pyspark.sql.functions import isnull
# Count null values in each column
null_counts = [df_redfin.where(isnull(col_name)).count() for col_name in df_redfin.columns]
null_counts

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0, 0, 0, 0, 0, 6187, 72370, 5761, 444284, 365221, 70762, 37380, 0]

In [12]:
# Display the results
for i, col_name in enumerate(df_redfin.columns):
    print(f"{col_name}: {null_counts[i]} null values")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

period_end: 0 null values
period_duration: 0 null values
city: 0 null values
state: 0 null values
property_type: 0 null values
median_sale_price: 6187 null values
median_ppsf: 72370 null values
homes_sold: 5761 null values
inventory: 444284 null values
months_of_supply: 365221 null values
median_dom: 70762 null values
sold_above_list: 37380 null values
last_updated: 0 null values

In [13]:
# Check for missing values in the entire DataFrame
remaining_count = df_redfin.na.drop().count()

print(f"Number of missing rows: {df_redfin.count() - remaining_count}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of missing rows: 535888

In [15]:
print(f"Total number of remaining rows: {remaining_count}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total number of remaining rows: 4986623

In [16]:
#remove na and count total number of remaining rows
df_redfin = df_redfin.na.drop()
print(f"Total number of rows: {df_redfin.count()}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total number of rows: 4986623

In [19]:
# Count null values in each column to confirm if we have removed all na
null_counts = [df_redfin.where(isnull(col_name)).count() for col_name in df_redfin.columns]
null_counts

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [20]:
from pyspark.sql.functions import year, month

#Extract year from period_end and save in a new column "period_end_yr"
df_redfin = df_redfin.withColumn("period_end_yr", year(col("period_end")))

#Extract month from period_end and save in a new column "period_end_month"
df_redfin = df_redfin.withColumn("period_end_month", month(col("period_end")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# Drop period_end and last_updated columns
df_redfin = df_redfin.drop("period_end", "last_updated")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
df_redfin.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+----------+----------+--------------------+-----------------+-----------------+----------+---------+----------------+----------+---------------+-------------+----------------+
|period_duration|      city|     state|       property_type|median_sale_price|      median_ppsf|homes_sold|inventory|months_of_supply|median_dom|sold_above_list|period_end_yr|period_end_month|
+---------------+----------+----------+--------------------+-----------------+-----------------+----------+---------+----------------+----------+---------------+-------------+----------------+
|             30|  Elbridge|  New York|Single Family Res...|         152000.0|96.08091024020227|         1|        4|             4.0|        10|            1.0|         2019|               7|
|             30|    Maumee|      Ohio|Single Family Res...|          75605.0|55.33251770348885|         6|       89|            14.8|        80|            0.0|         2012|               1|
|             30|Clear Lake|Washing

In [23]:
from pyspark.sql.functions import when

#let's map the month number to their respective month name.

df_redfin = df_redfin.withColumn("period_end_month", 
                   when(col("period_end_month") == 1, "January")
                   .when(col("period_end_month") == 2, "February")
                   .when(col("period_end_month") == 3, "March")
                   .when(col("period_end_month") == 4, "April")
                   .when(col("period_end_month") == 5, "May")
                   .when(col("period_end_month") == 6, "June")
                   .when(col("period_end_month") == 7, "July")
                   .when(col("period_end_month") == 8, "August")
                   .when(col("period_end_month") == 9, "September")
                   .when(col("period_end_month") == 10, "October")
                   .when(col("period_end_month") == 11, "November")
                   .when(col("period_end_month") == 12, "December")
                   .otherwise("Unknown")
                 )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
df_redfin.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+----------+----------+--------------------+-----------------+-----------------+----------+---------+----------------+----------+---------------+-------------+----------------+
|period_duration|      city|     state|       property_type|median_sale_price|      median_ppsf|homes_sold|inventory|months_of_supply|median_dom|sold_above_list|period_end_yr|period_end_month|
+---------------+----------+----------+--------------------+-----------------+-----------------+----------+---------+----------------+----------+---------------+-------------+----------------+
|             30|  Elbridge|  New York|Single Family Res...|         152000.0|96.08091024020227|         1|        4|             4.0|        10|            1.0|         2019|            July|
|             30|    Maumee|      Ohio|Single Family Res...|          75605.0|55.33251770348885|         6|       89|            14.8|        80|            0.0|         2012|         January|
|             30|Clear Lake|Washing

In [26]:
#let us write the final dataframe into our s3 bucket as a parquet file.
s3_bucket = "s3://transformeddata-redfin/redfin_data.parquet"
df_redfin.write.mode("overwrite").parquet(s3_bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…